<h1>Perform Federated Learning at Grid Platform </h1>
In this notebook, we will train a model using federated approach.

**NOTE:** At the time of running this notebook, we were running the grid components in background mode.  

**NOTE:**
Components:
 - Grid Gateway(http://localhost:8080)
 - Grid Node Bob (http://localhost:3000)
 - Grid Node Alice (http://localhost:3001)
 - Grid Node Bill (http://localhost:3002)

This notebook was made based on <a href="https://github.com/OpenMined/PySyft/blob/dev/examples/tutorials/Part%2010%20-%20Federated%20Learning%20with%20Secure%20Aggregation.ipynb">Part 10: Federated Learning with Encrypted Gradient Aggregation</a> tutorial

<h2>Import dependencies</h2>

In [1]:
import grid as gr
import syft as sy
import torch as th
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

/home/ionesio/workspace/dev/vev/lib/python3.7/site-packages/tensorflow-1.14.0-py3.7-linux-x86_64.egg/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ionesio/workspace/dev/vev/lib/python3.7/site-packages/tensorflow-1.14.0-py3.7-linux-x86_64.egg/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ionesio/workspace/dev/vev/lib/python3.7/site-packages/tensorflow-1.14.0-py3.7-linux-x86_64.egg/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,

<h2>Define Model</h2>

In [2]:
hook = sy.TorchHook(th)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 32)
        self.fc2 = nn.Linear(32, 24)
        self.fc3 = nn.Linear(24, 1)

    def forward(self, x):
        x = x.view(-1, 13)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01)

<h2> Connect to Grid Network</h2>

In [3]:
my_grid = gr.GridNetwork("http://localhost:5000")

<h2>Search a dataset</h2>

In [4]:
data = my_grid.search("#X", "#boston", "#housing")
target = my_grid.search("#Y", "#boston", "#housing")

In [5]:
data

[[(Wrapper)>[PointerTensor | me:37249732726 -> Bob:80786423983]],
 [(Wrapper)>[PointerTensor | me:9713249537 -> Alice:18146958968]],
 [(Wrapper)>[PointerTensor | me:75334145593 -> Bill:9562088539]]]

In [6]:
target

[[(Wrapper)>[PointerTensor | me:75640411060 -> Bob:81448698219]],
 [(Wrapper)>[PointerTensor | me:34800237337 -> Alice:25553850079]],
 [(Wrapper)>[PointerTensor | me:4445981120 -> Bill:65382525089]]]

<h2>Perform Train</h2>

In [7]:
def train(epoch):
    dataset_size = sum([ len(data[i][0]) for i in range(len(data))])
    model.train()
    for i in range(len(data)):
        worker = data[i][0].location
        model.send(worker)
        optimizer.zero_grad()
        pred = model(data[i][0])
        loss = F.mse_loss(pred.view(-1), target[i][0])
        loss.backward()
        optimizer.step()
        model.get()
        loss = loss.get()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * data[i][0].shape[0], dataset_size,
                       100. *  (i * data[i][0].shape[0]) / dataset_size, loss.item()))

for epoch in range(15):
    train(epoch)

Train Epoch: 0 [0/402 (0%)]	Loss: 600.671143
Train Epoch: 0 [134/402 (33%)]	Loss: 503.293976
Train Epoch: 0 [268/402 (67%)]	Loss: 593.302368
Train Epoch: 1 [0/402 (0%)]	Loss: 491.998962
Train Epoch: 1 [134/402 (33%)]	Loss: 335.492950
Train Epoch: 1 [268/402 (67%)]	Loss: 224.230820
Train Epoch: 2 [0/402 (0%)]	Loss: 228.273834
Train Epoch: 2 [134/402 (33%)]	Loss: 424.945129
Train Epoch: 2 [268/402 (67%)]	Loss: 468.684082
Train Epoch: 3 [0/402 (0%)]	Loss: 310.441101
Train Epoch: 3 [134/402 (33%)]	Loss: 166.916702
Train Epoch: 3 [268/402 (67%)]	Loss: 114.200340
Train Epoch: 4 [0/402 (0%)]	Loss: 147.395859
Train Epoch: 4 [134/402 (33%)]	Loss: 320.018402
Train Epoch: 4 [268/402 (67%)]	Loss: 288.444458
Train Epoch: 5 [0/402 (0%)]	Loss: 132.126160
Train Epoch: 5 [134/402 (33%)]	Loss: 35.871387
Train Epoch: 5 [268/402 (67%)]	Loss: 30.129076
Train Epoch: 6 [0/402 (0%)]	Loss: 21.753355
Train Epoch: 6 [134/402 (33%)]	Loss: 20.651144
Train Epoch: 6 [268/402 (67%)]	Loss: 27.102346
Train Epoch: 7 [0/

<h2>Disconnect Grid Nodes</h2>

In [8]:
my_grid.disconnect_nodes()